In [45]:
import pandas as pd
import numpy as np

In [2]:
dt_2014 = pd.read_csv("datasets/constituency_wise_results_2014.csv")
dt_2019 = pd.read_csv("datasets/constituency_wise_results_2019.csv")

In [3]:
dt_2014.sample(5)

,state,pc_name,candidate,sex,age,category,party,party_symbol,general_votes,postal_votes,total_votes,total_electors
2293,Karnataka,Raichur,M NAGARAJ,M,29.0,ST,CPIM,Saw,6938,0,6938,1661606
6168,Uttar Pradesh,Agra,AMAR SINGH,M,38.0,SC,IND,Coconut,2450,3,2453,1814739
7036,Uttar Pradesh,Azamgarh,None of the Above,NaN,NaN,NaN,NOTA,NOTA,5660,0,5660,1703121
7785,Jharkhand,Rajmahal,SUNIRAM HEMBROM,M,45.0,ST,IND,Cot,6460,0,6460,1353467
6015,Uttar Pradesh,Amroha,KANWAR SINGH TANWAR,M,52.0,GEN,BJP,Lotus,528796,84,528880,1544245


In [4]:
def data_corrector(dataframe):
    dataframe["pc_name"] = np.where((dataframe["state"] == "Himachal Pradesh") & (dataframe["pc_name"] == "Hamirpur"), "Hamirpur (Himachal Pradesh)",
    np.where((dataframe["state"] == "Bihar") & (dataframe["pc_name"] == "Aurangabad"), "Aurangabad (Bihar)",
    np.where((dataframe["state"] == "Bihar") & (dataframe["pc_name"] == "Maharajganj"), "Maharajganj (Bihar)",
    np.where((dataframe["state"] == "Uttar Prades") & (dataframe["pc_name"] == "Hamirpur"), "Hamirpur (Uttar Prades)",
    np.where((dataframe["state"] == "Uttar Prades") & (dataframe["pc_name"] == "Maharajganj"), "Maharajganj (Uttar Prades)",
    np.where((dataframe["state"] == "Maharashtra") & (dataframe["pc_name"] == "Aurangabad"), "Aurangabad (Maharashtra)",dataframe["pc_name"]))))))

In [5]:
data_corrector(dt_2014)
data_corrector(dt_2019)

# 1

In [6]:
group = dt_2014.groupby("pc_name")

In [7]:
temp_df1 = group["total_votes"].sum().to_frame()

In [8]:
temp_df2 = group["total_electors"].mean().astype(int).to_frame()

In [9]:
df = pd.merge(temp_df1,temp_df2, on = "pc_name")

In [10]:
df["Turnout Ratio"] = round((df["total_votes"] / df["total_electors"])*100,1)

In [54]:
print("top 5 constituencies of 2014 in terms of voter turnout ratio.")
df.sort_values("Turnout Ratio",ascending = False).head(5)

top 5 constituencies of 2014 in terms of voter turnout ratio.


,total_votes,total_electors,Turnout Ratio
state,,,
Lakshadweep,47009,55189,85.2
Nagaland,1006357,1213777,82.9
Manipur,1617330,1959563,82.5
Tripura,2153172,2614718,82.3
West Bengal,57206976,70001284,81.7


In [47]:
print("bottom 5 constituencies of 2014 in terms of voter turnout ratio.")
df.sort_values("Turnout Ratio").head(5)

bottom 5 constituencies of 2014 in terms of voter turnout ratio.


,total_votes,total_electors,Turnout Ratio
state,,,
Jammu & Kashmir,3552622,7922538,44.8
Bihar,40806661,71216290,57.3
Uttar Pradesh,86481398,146134603,59.2
NCT OF Delhi,8679012,14327649,60.6
Maharashtra,54054245,88676946,61.0


# 2

In [13]:
group = dt_2019.groupby("pc_name")

In [14]:
temp_df1 = group["total_votes"].sum().to_frame()

In [15]:
temp_df2 = group["total_electors"].mean().astype(int).to_frame()

In [16]:
df = pd.merge(temp_df1,temp_df2, on = "pc_name")

In [17]:
df["Turnout Ratio"] = round((df["total_votes"] / df["total_electors"])*100,1)

In [48]:
print("top 5 constituencies of 2019 in terms of voter turnout ratio.")
df.sort_values("Turnout Ratio",ascending = False).head(5)

top 5 constituencies of 2019 in terms of voter turnout ratio.


,total_votes,total_electors,Turnout Ratio
state,,,
Lakshadweep,47009,55189,85.2
Nagaland,1006357,1213777,82.9
Manipur,1617330,1959563,82.5
Tripura,2153172,2614718,82.3
West Bengal,57206976,70001284,81.7


In [49]:
print("bottom 5 constituencies of 2019 in terms of voter turnout ratio.")
df.sort_values("Turnout Ratio").head(5)

bottom 5 constituencies of 2019 in terms of voter turnout ratio.


,total_votes,total_electors,Turnout Ratio
state,,,
Jammu & Kashmir,3552622,7922538,44.8
Bihar,40806661,71216290,57.3
Uttar Pradesh,86481398,146134603,59.2
NCT OF Delhi,8679012,14327649,60.6
Maharashtra,54054245,88676946,61.0


# 3

In [20]:
group = dt_2014.groupby("pc_name")

In [21]:
temp_df1 = group["total_votes"].sum().to_frame()

In [22]:
temp_df2 = group["total_electors"].mean().astype(int).to_frame()

In [23]:
temp_df3 = group["state"].apply(lambda x: x.unique()[0]).to_frame()

In [24]:
df = pd.merge(temp_df1,temp_df2, on = "pc_name")

In [25]:
df = pd.merge(df,temp_df3, on = "pc_name")

In [26]:
df.set_index("state", inplace = True)

In [27]:
df = df.groupby("state").sum()

In [28]:
df["Turnout Ratio"] = round((df["total_votes"] / df["total_electors"])*100,1)

In [50]:
print("top 5 states of 2014 in terms of voter turnout ratio.")
df.sort_values("Turnout Ratio",ascending = False).head(5)

top 5 states of 2014 in terms of voter turnout ratio.


,total_votes,total_electors,Turnout Ratio
state,,,
Lakshadweep,47009,55189,85.2
Nagaland,1006357,1213777,82.9
Manipur,1617330,1959563,82.5
Tripura,2153172,2614718,82.3
West Bengal,57206976,70001284,81.7


In [51]:
print("bottom 5 states of 2014 in terms of voter turnout ratio.")
df.sort_values("Turnout Ratio").head(5)

bottom 5 states of 2014 in terms of voter turnout ratio.


,total_votes,total_electors,Turnout Ratio
state,,,
Jammu & Kashmir,3552622,7922538,44.8
Bihar,40806661,71216290,57.3
Uttar Pradesh,86481398,146134603,59.2
NCT OF Delhi,8679012,14327649,60.6
Maharashtra,54054245,88676946,61.0


# 4

In [31]:
group = dt_2019.groupby("pc_name")

In [32]:
temp_df1 = group["total_votes"].sum().to_frame()

In [33]:
temp_df2 = group["total_electors"].mean().astype(int).to_frame()

In [34]:
temp_df3 = group["state"].apply(lambda x: x.unique()[0]).to_frame()

In [35]:
df = pd.merge(temp_df1,temp_df2, on = "pc_name")

In [36]:
df = pd.merge(df,temp_df3, on = "pc_name")

In [37]:
df.set_index("state", inplace = True)

In [38]:
df = df.groupby("state").sum()

In [39]:
df["Turnout Ratio"] = round((df["total_votes"] / df["total_electors"])*100,1)

In [52]:
print("top 5 states of 2019 in terms of voter turnout ratio.")
df.sort_values("Turnout Ratio",ascending = False).head(5)

top 5 states of 2019 in terms of voter turnout ratio.


,total_votes,total_electors,Turnout Ratio
state,,,
Lakshadweep,47009,55189,85.2
Nagaland,1006357,1213777,82.9
Manipur,1617330,1959563,82.5
Tripura,2153172,2614718,82.3
West Bengal,57206976,70001284,81.7


In [53]:
print("bottom 5 states of 2019 in terms of voter turnout ratio.")
df.sort_values("Turnout Ratio").head(5)

bottom 5 states of 2019 in terms of voter turnout ratio.


,total_votes,total_electors,Turnout Ratio
state,,,
Jammu & Kashmir,3552622,7922538,44.8
Bihar,40806661,71216290,57.3
Uttar Pradesh,86481398,146134603,59.2
NCT OF Delhi,8679012,14327649,60.6
Maharashtra,54054245,88676946,61.0
